In [1]:
import seaborn as sns
import math
import numpy as np
from scipy.stats import norm, t  # таблица накопленного нормраспределения z
sns.set_theme()

# стандартные параметры нормального распределения
def norm_mu_func(lst): 
    return sum(lst) / len(lst)

def norm_dispersion_func(data):
    mean = norm_mu_func(data)
    d_sum = 0
    for i in data:
        d_sum = d_sum + (mean - i)**2
    return d_sum / len(data)

def norm_sigma_func(data):
    return norm_dispersion_func(data)**0.5

def norm_dist_graph(data, X=None):
    M = norm_mu_func(data)
    sigma = norm_sigma_func(data)
    y = lambda data: list(map(lambda x: \
               math.e**(-((x-M)**2)/(2*sigma**2)) / (sigma  * (2*math.pi)**0.5), \
               data))
    if X:
        return y([X])
    else:
        plt = sns.lineplot(x=data, y=y(data))
        plt.axvline(M)
        plt.axvline(M-sigma)
        plt.axvline(M+sigma)

def norm_get_z(to_find, sigma, M):
    return (to_find - M) / sigma

def norm_z_to_probability(z):
    '''
    вытащить значение из таблицы накопленного нормального распредееления по z
    '''
    return norm.cdf(z)


# стандартные параметры равномерного распределения
def ravn_mu_func(data): 
    return (max(data)+min(data))/2

def ravn_dispersion_func(data):
    return (max(data)-min(data))**2 / 12

def ravn_sigma_func(data):
    return ravn_dispersion_func(data)**0.5

def ravn_dist_graph(data, X=None):
    M = ravn_mu_func(data)
    sigma = ravn_sigma_func(data)
    a = min(data)
    b = max(data)
    y = lambda data: list(map(lambda x: \
                              (x-a) / (b-a), \
                              data))
    for ind, i in enumerate(y(data)):
        if i < 0:
            y[ind] = 0
        if i > 1:
            y[ind] = 1
    if X:
        return y([X])
    else:
        return sns.lineplot(x=x, y=y(x))

In [2]:
def z_criteriy(M, a, sigma, n):
    Z = -norm.ppf(a / 2)
    interval = Z * (sigma / n**0.5)
    return [M - interval, M + interval]


def t_criteriy(data, M, a, n):
    '''критерий-стьюдента'''
    T = -t.ppf(a / 2, df=n-1)
    sigma = np.std(data, ddof=1) # несмещенная сигма (n-1)
    interval = T * (sigma / n**0.5)
    return [M - interval, M + interval]

# (1)
## Известно, что генеральная совокупность распределена нормально со средним квадратическим отклонением, равным 16. 
## Найти доверительный интервал для оценки математического ожидания a с надежностью 0.95, если выборочная средняя M = 80, а объем выборки n = 256.
*решать через доверительный интервал*

In [3]:
sigma = 16
alfa  = 1 - 0.95
M     = 80
n     = 256

In [4]:
# так как ср.кв.откл. известно, то для расчета будем использовать z-критерий
f'С 95% вероятностью матожидание находится в интервале {z_criteriy(M, alfa, sigma, n)}'

'С 95% вероятностью матожидание находится в интервале [78.04003601545995, 81.95996398454005]'

# (2)
## В результате 10 независимых измерений некоторой величины X, выполненных с одинаковой точностью, получены опытные данные: 6.9, 6.1, 6.2, 6.8, 7.5, 6.3, 6.4, 6.9, 6.7, 6.1 
## Предполагая, что результаты измерений подчинены нормальному закону распределения вероятностей, оценить истинное значение величины X при помощи доверительного интервала, покрывающего это значение с доверительной вероятностью 0,95.
*решать через доверительный интервал*

In [5]:
x = [6.9, 6.1, 6.2, 6.8, 7.5, 6.3, 6.4, 6.9, 6.7, 6.1]
alfa  = 1 - 0.95

In [6]:
# так как ср.кв.откл. НЕ известно, то для расчета будем использовать student-критерий
M = norm_mu_func(x)
n = len(x)
f'С 95% вероятностью матожидание находится в интервале {t_criteriy(x, M, alfa, n)}'

'С 95% вероятностью матожидание находится в интервале [6.267515851415712, 6.912484148584286]'

# (3)
## Утверждается, что шарики для подшипников, изготовленные автоматическим станком, имеют средний диаметр 17 мм.
## Используя односторонний критерий с α=0,05, проверить эту гипотезу, если в выборке из n=100 шариков средний диаметр оказался равным 17.5 мм, а дисперсия известна и равна 4 кв.мм.
*решать через тестирование гипотезы*

In [7]:
m_0 = 17
alfa = 0.05
n = 100
m_1 = 17.5
D = 4

In [8]:
sigma = D**0.5
# так как ср.кв.откл. известно, то для расчета будем использовать z-критерий
# гипотеза: m_0 < m_1
Z_alfa = norm.ppf(1-alfa) # Z на уровне alfa
Z_test = (m_1 - m_0) / (sigma/n**0.5)
print(Z_alfa, Z_test)

1.6448536269514722 2.5


In [9]:
if Z_test > Z_alfa:
    print(f'Z_test > Z_alfa => гипотеза верна при уровне значимости {alfa*100}%. Принимаем 1 гипотезу.')
elif Z_test == Z_alfa:
    print(f'Z_test = Z_alfa => гипотеза не верна при уровне значимости {alfa*100}%.')
else:
    print(f'Z_test < Z_alfa => гипотеза не верна при уровне значимости {alfa*100}%.')

Z_test > Z_alfa => гипотеза верна при уровне значимости 5.0%. Принимаем 1 гипотезу.


# (4)
## Продавец утверждает, что средний вес пачки печенья составляет 200 г. Из партии извлечена выборка из 10 пачек. Вес каждой пачки составляет: 202, 203, 199, 197, 195, 201, 200, 204, 194, 190. Известно, что их веса распределены нормально. 
## Верно ли утверждение продавца, если учитывать, что доверительная вероятность равна 99%?
*решать через тестирование гипотезы*

In [10]:
m_0 = 200
n = 10
x = [202, 203, 199, 197, 195, 201, 200, 204, 194, 190]
alfa = 0.01

In [11]:
# ср.кв.отклонение неизвестно - используем t-критерий
# гипотеза: m_0 != m_1
m_1 = norm_mu_func(x)
T_alfa_right = t.ppf(1 - alfa/2, df=n-1) # правое значение t
T_alfa_left  = t.ppf(alfa/2,     df=n-1) # левое  значение t
T_alfa_left, T_alfa_right

(-3.24983554401537, 3.2498355440153697)

In [12]:
sigma = np.std(x)
T_test = (m_1 - m_0) / (sigma/n**0.5)
T_test

-1.1227217828476797

In [13]:
print('T_test находится в рамках значений T нормального распределения нулевой гипотезы при уровне значимости ' + str(alfa*100) + '%')
print('=> гипотеза неверна (принимаем нулевую гипотезу)')

T_test находится в рамках значений T нормального распределения нулевой гипотезы при уровне значимости 1.0%
=> гипотеза неверна (принимаем нулевую гипотезу)
